In [1]:
import pathlib
import sys
import os
parser_dir = pathlib.Path(os.path.dirname(os.path.abspath("__file__"))) / 'java-universal-parser' / 'parser'
sys.path.append(str(parser_dir))

In [2]:
from python3.JavaLexer import JavaLexer
from python3.JavaParser import JavaParser

from antlr4 import InputStream, CommonTokenStream

In [3]:
from antlr4.error.ErrorListener import ErrorListener

class MyErrorListener( ErrorListener ):

    def __init__(self):
        super(MyErrorListener, self).__init__()

    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        #print("line " + str(line) + ":" + str(column) + " " + msg, file=sys.stderr)
        raise Exception("Oh no!!", line, column)


In [4]:
def check_syntax(code):
    try:
        input_stream = InputStream(code)
        lexer = JavaLexer(input_stream)
        lexer.removeErrorListeners() # remove default error listener
        lexer.addErrorListener( MyErrorListener() )
        stream = CommonTokenStream(lexer)
        parser = JavaParser(stream)
        parser.removeErrorListeners()
        parser.addErrorListener( MyErrorListener() )
        parser.compilationUnit()
    except Exception as e:
        line = e.args[1]
        return False, line
    return True, None

In [5]:
inner_brace = "\n    }"
outer_brace = "\n\n}"


def get_brace_count(code, start_level=0):
    count = start_level
    for char in code:
        if char == "{":
            count += 1
        if char == "}":
            count -= 1
    return count

def balance_braces(code, start_level=0):
    count = get_brace_count(code, start_level)
    if count == 0:
        #print("Well formed code",)
        return code
    
    lines = code.splitlines(keepends=True)
    #print("Lines: ", len(lines))
    for line in reversed(range(len(lines))):
        candidate = "".join(lines)
        count = get_brace_count(candidate, start_level)
        #print("Count: ", count)
        #print("Line idx: ", line)
        if count == 0:
            #print("0")
            return candidate
        elif count == 1:
            #print("1")
            candidate += outer_brace
            return candidate
        elif count == 2:
            #print("2")
            candidate += inner_brace
            candidate += outer_brace
            return candidate
        lines.pop()

    return False


In [6]:
def well_formed_code(code):
    success, bad_line = check_syntax(code)
    lines = code.splitlines(keepends=False) 
    if not success:
        # Trim the code to the bad line
        lines = lines[:bad_line-1]

        if len(lines) <= 1:
            return False, None

        else:
            balanced_code = balance_braces("\n".join(lines))
            success, bad_line = check_syntax(balanced_code)

            if not success:
                return well_formed_code("\n".join(lines))
            else:
                return True, balanced_code

    else:
        return True, code

# fixed

In [7]:
from tqdm.auto import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

def well_formed_fix(code):
    prefix = """class dummyTest {
        @Test
        public void dummy() {
    """
    code = prefix + code
    try:
        status, res = well_formed_code(code)
    
        if status:
            res_trimmed = res.removeprefix(prefix)
            return res_trimmed
        else:
            return None
    except Exception as e:
            print("Error: ", e)
            return None


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [8]:
import os
import pandas as pd
from pathlib import Path
import re


def fix_data(dataset, method, model):
    regex = r"^(\d+)-of-(\d+).test.jsonl$"
    path= Path("../data/" + str(dataset) +  "/generated/" + str(method) +  "/" + str(model))
    files = os.listdir(path)

    
    dfs = []
    for file in files:
        matches = re.search(regex, file)
        if matches:
            df = pd.read_json(path / file, orient='records', lines=True, dtype=False)
            dfs.append(df)


    df = pd.concat(dfs, axis=0).set_index('id')
    df.sort_index(inplace=True)
    df = df[~df.index.duplicated(keep='first')]
    df = df[["prompt", "reference", "prediction"]]

    total = df.shape[0]
    
    df["prediction"] = df["prediction"].parallel_apply(well_formed_fix)
    
    # remove "}\n\n}" and none values
    fixed_df = df[df["prediction"]!="}\n\n}"]
    fixed_df = fixed_df[fixed_df["prediction"].notnull()]
    
    num_fixed = fixed_df.shape[0]
    
    # save fixed predictions as jsonl
    
    save_path= Path("../data/" + str(dataset) +  "/fixed/" + str(method) +  "/" + str(model))
    save_path.mkdir(parents=True, exist_ok=True)
    
    fixed_df.reset_index().to_json(save_path / "00001-of-00001.jsonl", orient='records', lines=True)
    return (num_fixed, total)


In [9]:
import os
from pathlib import Path
from tqdm.auto import tqdm
import os

root, dirs, files = next(os.walk("../data"))
datasets = dirs

for dataset in datasets:
    dataset_path = Path(root, dataset, "generated")
    
    paths = []
    for d_root, dirnames, filenames in os.walk(dataset_path):
        # only keep "output" directories
        path = Path(d_root)
        #print(path.parts)
        if len(path.parts) == 7:
            paths.append(path)

    paths = list(set(paths))

    valid_syntax_data = {}
    for path in tqdm(paths):
        path = path.parts
        method = path[4]
        model = path[5] + "/" + path[6]
        print(dataset, method, model)
        fixed, total = fix_data(dataset, method, model)

        valid_syntax_data.setdefault(model, {})
        percentage = fixed / total if total > 0 else 0
        valid_syntax_data[model][method] = percentage

    valid_df = pd.DataFrame(valid_syntax_data)
    valid_df.T.to_csv(Path(root, dataset, "valid_syntax.csv"), index_label = 'model')

  0%|          | 0/50 [00:00<?, ?it/s]

humaneval-x prompt-tuning Salesforce/codegen2-16B_P


humaneval-x prompt-tuning Salesforce/codegen2-7B_P


humaneval-x fine-tuning bigcode/starcoder2-3b


humaneval-x fine-tuning metal-llama/CodeLlama-7b-hf


...

methods2test_runnable prompt-tuning Salesforce/codegen2-1B_P


methods2test_runnable fine-tuning bigcode/starcoder2-3b


methods2test_runnable fine-tuning Salesforce/codegen2-3_7B_P


Error:  tuple index out of range
Error:  tuple index out of range
Error:  tuple index out of range
Error:  tuple index out of range
Error:  tuple index out of range
Error:  tuple index out of range
Error:  tuple index out of range
methods2test_runnable ia3 meta-llama/CodeLlama-7b-hf


methods2test_runnable lora bigcode/starcoder2-7b
